In [1]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
import scipy.stats as stats
from matplotlib import gridspec
import aplpy as apl #The Astronomy Plotting Library for python
import astropy
from astropy.stats import sigma_clip
from astropy.modeling import functional_models, models, fitting
import astropy.units as U
from astropy.coordinates import ICRS, Galactic, FK4, FK5, Angle, Latitude, Longitude
import astropy.constants as C
from astropy import wcs
import astropy.io.fits as fits
from astropy.io import ascii
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.stats import signal_to_noise_oir_ccd as SNRas
#import spectral_cube as SC
#from spectral_cube import SpectralCube as sc
from astropy.wcs import WCS
matplotlib.use('Agg')
import matplotlib.cm as cm
#import astrometry as ast
import pyfits
import requests
import json
import os
from sklearn import datasets, linear_model
from scipy.optimize import curve_fit
import numpy.ma as ma
from astropy.table import Table, Column
import pyextract
import sewpy
import pandas as pd
from astropy import units as u

/home/lynge/anaconda2/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/lynge/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/lynge/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/lynge/anaconda2

In [2]:
sta1_RA_WORLD = SkyCoord('13:49:06.5 -30:20:36', unit=(u.hourangle, u.deg)).ra.degree 
sta1_DEC_WORLD = SkyCoord('13:49:06.5 -30:20:36', unit=(u.hourangle, u.deg)).dec.degree

sta2_RA_WORLD = SkyCoord('13:49:10 -30:16:21', unit=(u.hourangle, u.deg)).ra.degree
sta2_DEC_WORLD = SkyCoord('13:49:10 -30:16:21', unit=(u.hourangle, u.deg)).dec.degree

sta3_RA_WORLD = SkyCoord('13:49:24.3 -30:20:20', unit=(u.hourangle, u.deg)).ra.degree #TYC 8480-374-1
sta3_DEC_WORLD = SkyCoord('13:49:24.3 -30:20:20', unit=(u.hourangle, u.deg)).dec.degree

sky_RA_WORLD = SkyCoord('13:49:22 -30:17:04', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
sky_DEC_WORLD = SkyCoord('13:49:22 -30:17:04', unit=(u.hourangle, u.deg)).dec.degree


k_g = 0.16
k_r = 0.08
k_i = 0.02
k_z = 0.01

mag_refsta1_g = 10.166
mag_refsta1_r = 10.229
mag_refsta1_i = 10.347
mag_refsta1_z = 10.456

refsta1_RA = SkyCoord('08:52:34 -00:39:49', unit=(u.hourangle, u.deg)).ra.degree #PG1323_086_10_07_17
refsta1_DEC = SkyCoord('08:52:34 -00:39:49', unit=(u.hourangle, u.deg)).dec.degree 


In [3]:
QUASAR1_g = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-IC4329A/g/',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-IC4329A/g/')]

refsta1_g = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/g',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/g')]

QUASAR1_r = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-IC4329A/r/',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-IC4329A/r/')]

refsta1_r = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/r',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/r')]

QUASAR1_i = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-IC4329A/i/',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-IC4329A/i/')]

refsta1_i = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/i',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/i')]

QUASAR1_z = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-IC4329A/z/',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-IC4329A/z/')]

refsta1_z = [os.path.join('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/z',f) \
        for f in os.listdir('/media/lynge/LACIE/Calibration/08-06-16-SA100-241/z')]


In [4]:
def dist(mid,end):
    return np.sqrt((mid[0] - end[0])**2 + (mid[1] - end[1])**2)

In [5]:
def flux_cal(A_mag,B_mag,B_flux):
    '''Flux of A found through magnitude and flux of B'''
    return B_flux*10**((B_mag-A_mag)/2.5)

In [6]:
def Basic(quasar,header):
    '''Provides a basic list of neccesities from the fits header file for use in the later \
    flux determination, it does not serve an individual purpose beyound that'''
    AGNdata = quasar #np.rot90(np.rot90(np.fliplr(quasar)))
    #header = data[0].header
    #print header
    RAstd = header['CRVAL1']
    DECstd = header['CRVAL2']
    #print RAstd, DECstd
    pixspa = header['PIXSCALE']/(3600.)
    #print pixspa
    RAstdpix = header['CRPIX1']
    DECstdpix = header['CRPIX2']
    exptime = 10 # header['EXPTIME']
    return header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime

In [7]:
def curve(quasar,header,w,center,apparature):
    '''Determines the FLUX of the stellar object given the the numpy array of the image, \
    the header of the fits file, the astropy coordinate representation, \
    the object position and the apparature of interest'''
    header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime = Basic(quasar,header)
    #AGNdata = np.swapaxes(AGNdata,0,1)
    y, x = np.ogrid[0:header['NAXIS1'],0:header['NAXIS1']]
    #w = np.swapaxes(w,0,1)
    #F_AGN = np.zeros((512,512,3)) #((int(2*apparature/pixspa)+1,int(2*apparature/pixspa)+1,3))
    #print center
    x1 = w.all_world2pix(center[0],center[1],0)
    x1,y1 = (x1[0]),(x1[1])
    try:
        mask = ((y-y1)**2 + (x-x1)**2) > (apparature/float(pixspa))**2
        AGNdata[mask] = float(0)
        mask = AGNdata == 0.
        rows = np.flatnonzero((~mask).sum(axis=1))
        cols = np.flatnonzero((~mask).sum(axis=0))
        AGNdata = AGNdata[rows.min():rows.max()+1, cols.min():cols.max()+1]
        #print apparature/float(pixspa)
    except:
        AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    #if np.sum(AGNdata) < 500:
    #    AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    return AGNdata/float(exptime), x1, y1


In [8]:
def comp1(qua,sta):
    '''Code to determine the relation between the AGN and the Stellar flux as determined by either me \
    or by the SExtractor program respectively'''
    comp = qua/sta
    return comp

In [9]:
def SNR(data1,data2,header,ap1,ap2):
    npix1 = np.pi*ap1**2/((header['PIXSCALE']/3600.)**2)
    npix2 = np.pi*ap2**2/((header['PIXSCALE']/3600.)**2)
    #data2_1 = ((data2-data1)/npix2)*npix1
    data1_2 = data1/10.
    data2_2 = data2/10.
    return SNRas(10.,data1_2,abs(data2_2),0,header['RON'],npix1,gain=header['GAIN'])

In [10]:
def SXtable(quas,aperture):
    header = pyfits.getheader(quas)
    phot_app = aperture*3600/header['PIXSCALE']
    np.set_printoptions(threshold=np.inf)
    sew = sewpy.SEW(params=['XMODEL_WORLD', 'YMODEL_WORLD', 'X2MODEL_WORLD', 'Y2MODEL_WORLD', 
                            'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_RADIUS(1)', 'FLUX_APER','ELLIPTICITY'],
                    config={"DETECT_THRESH":5, 'DETECT_MINAREA':20, 'DETECT_MAXAREA':30000, 
                            "PHOT_FLUXFRAC":"0.5", 'PHOT_APERTURES':2*phot_app, 
                            'PIXEL_SCALE':header['PIXSCALE']}
                    , sexpath='/usr/bin/sextractor')
    out = sew(quas)
    return out

In [11]:
def flux(quas,aperture,main_RA_WORLD,main_DEC_WORLD):
    header = pyfits.getheader(quas)
    RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[0]
    DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[1]
    output = float('nan')
    if 15 < RAmain < header['NAXIS1'] - 15 and 15 < DECmain < header['NAXIS2'] - 15: 
        out = SXtable(quas,aperture)
        #print out['table']['XMODEL_IMAGE']
        #print RAmain
        RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[0]
        DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[1]
        #print out['table']['XMODEL_IMAGE'], out['table']['YMODEL_IMAGE']
        #print RAmain, DECmain

        for j in range(len(out['table']['XMODEL_IMAGE'])):
            #output['time'][i] = header['MJD-OBS']
            if RAmain - 20. < out['table']['XMODEL_IMAGE'][j] <  RAmain + 20. and \
            DECmain - 20. < out['table']['YMODEL_IMAGE'][j] < DECmain + 20.:
                #print 'YES'
                output = out['table']['FLUX_APER'][j]
                break
    else:
        output = float('nan')
        #print 'Done'
    return output


In [12]:
def findmag(start,compare1,RA_star,DEC_star,RA_com,DEC_com,mag_com,AIR_EXTINCTION):
    print mag_com
    aperture1 = 0.0012
    #201.5441,-8.8601
    com_flu = []
    com_AIRMASS = []
    com_EXPTIME = []
    star_flu = []
    star_AIRMASS = []
    star_EXPTIME = []
    for i in range(len(compare1)):
        print 'Frame nr. ', i, compare1[i]
        header = pyfits.getheader(compare1[i])
        a1 = flux(compare1[i],2*aperture1,RA_com,DEC_com)/float(header['EXPTIME'])
        print header['EXPTIME']
        if a1 > 0:
            #print a1
            com_flu.append(a1), com_AIRMASS.append(header['AIRMASS']*AIR_EXTINCTION), com_EXPTIME.append(header['EXPTIME'])
        else:
            #print a1
            com_flu.append(float('nan')), com_AIRMASS.append(float('nan')), com_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    for i in range(len(start)):
        print 'Frame nr. ', i, start[i]
        header = pyfits.getheader(start[i])
        a1 = flux(start[i],2*aperture1,RA_star,DEC_star)/float(header['EXPTIME'])
        #print header['EXPTIME']
        if a1 > 0:
            #print a1
            star_flu.append(a1), star_AIRMASS.append(header['AIRMASS']*AIR_EXTINCTION), star_EXPTIME.append(header['EXPTIME'])
        else:
            star_flu.append(float('nan')), star_AIRMASS.append(float('nan')), star_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    com_flu = np.array(com_flu)
    #print com_flu
    com_AIRMASS = np.array(com_AIRMASS)
    com_EXPTIME = np.array(com_EXPTIME)
    star_flu = np.array(star_flu)
    #print star_flu
    star_AIRMASS = np.array(star_AIRMASS)
    star_EXPTIME = np.array(star_EXPTIME)
    com_flu = com_flu#/com_EXPTIME
    star_flu = star_flu#/star_EXPTIME
    com_flux = np.nanmean(com_flu)
    com_AIRMASS_num = np.nanmean(com_AIRMASS)
    star_AIRMASS_num = np.nanmean(star_AIRMASS)
    #print star_AIRMASS_num
    mag1 = (star_AIRMASS_num-com_AIRMASS_num) + mag_com - 2.5*np.log10(star_flu/com_flux)
    mag = mag1
    #print (star_AIRMASS_num-com_AIRMASS_num)
    #print mag_com, com_flux, np.average(com_flu)
    #print mag
    return mag

In [55]:
com1_1 = findmag(QUASAR1_g,refsta1_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)
com2_1 = findmag(QUASAR1_g,refsta1_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)
com3_1 = findmag(QUASAR1_g,refsta1_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)



10.166
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/g/IMG2016158URh009.fits
40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/g/IMG2016158URh010.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/g/IMG2016158URh013.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/g/IMG2016158URh014.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/g/IMG2016158URs011.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/g/IMG2016158URs012.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/g/IMG2016158URs015.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/g/IMG2016158URs016.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/g/IMG2016158URs102.fits
Frame nr.  1 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/g/IMG2016158URs103.fits
Frame nr.  2 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/g/IMG2016158URs101.fits
10

In [56]:
print 'g-filter'
print 'Star 1'
print com1_1
print np.nanmean((com1_1)), np.nanstd((com1_1))

print 'Star 2'
print com2_1
print np.nanmean((com2_1)), np.nanstd((com2_1))

print 'Star 3'
print com3_1
print np.nanmean((com3_1)), np.nanstd((com3_1))

g-filter
Star 1
[13.72310189 13.7304397  13.71711243]
13.7235513392078 0.005450108367667869
Star 2
[15.96089073 15.93236005 15.96046294]
15.951237905835535 0.013349799045560044
Star 3
[15.44114078 15.4818575  15.47287619]
15.465291489757627 0.017466319007235352


In [57]:
com1_1 = findmag(QUASAR1_r,refsta1_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)
com2_1 = findmag(QUASAR1_r,refsta1_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)
com3_1 = findmag(QUASAR1_r,refsta1_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)



10.229
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/r/IMG2016158BRh009.fits
40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/r/IMG2016158BRh010.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/r/IMG2016158BRh013.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/r/IMG2016158BRh014.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/r/IMG2016158BRs011.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/r/IMG2016158BRs012.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/r/IMG2016158BRs015.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/r/IMG2016158BRs016.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/r/IMG2016158BRs102.fits
Frame nr.  1 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/r/IMG2016158BRs103.fits
Frame nr.  2 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/r/IMG2016158BRs101.fits
10

In [58]:
print 'r-filter'
print 'Star 1'
print com1_1
print np.nanmean((com1_1)), np.nanstd((com1_1))

print 'Star 2'
print com2_1
print np.nanmean((com2_1)), np.nanstd((com2_1))

print 'Star 3'
print com3_1
print np.nanmean((com3_1)), np.nanstd((com3_1))

r-filter
Star 1
[13.11173469 13.10886648 13.11062584]
13.110409001492366 0.0011809380683228841
Star 2
[15.54223056 15.49851126 15.46127555]
15.500672455685153 0.0330850551670884
Star 3
[15.02366743 15.0165033  15.02565294]
15.021941222522821 0.003929699688318341


In [59]:
com1_1 = findmag(QUASAR1_i,refsta1_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)
com2_1 = findmag(QUASAR1_i,refsta1_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)
com3_1 = findmag(QUASAR1_i,refsta1_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)



10.347
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/i/IMG2016158ULh009.fits
40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/i/IMG2016158ULh010.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/i/IMG2016158ULh013.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/i/IMG2016158ULh014.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/i/IMG2016158ULs011.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/i/IMG2016158ULs012.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/i/IMG2016158ULs015.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/i/IMG2016158ULs016.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/i/IMG2016158ULs102.fits
Frame nr.  1 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/i/IMG2016158ULs103.fits
Frame nr.  2 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/i/IMG2016158ULs101.fits
10

In [60]:
print 'i-filter'
print 'Star 1'
print com1_1
print np.nanmean((com1_1)), np.nanstd((com1_1))

print 'Star 2'
print com2_1
print np.nanmean((com2_1)), np.nanstd((com2_1))

print 'Star 3'
print com3_1
print np.nanmean((com3_1)), np.nanstd((com3_1))

i-filter
Star 1
[12.92531982 12.93332356 12.92812788]
12.928923753464538 0.0033156253143878187
Star 2
[15.2537722  15.2784724  15.25454767]
15.262264089351314 0.011465377563467702
Star 3
[14.86996455 14.89481957 14.88305275]
14.88261228974757 0.010151800248950217


In [13]:
com1_1 = findmag(QUASAR1_z,refsta1_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)
com2_1 = findmag(QUASAR1_z,refsta1_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)
com3_1 = findmag(QUASAR1_z,refsta1_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)



10.456
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLh009.fits


/home/lynge/anaconda2/lib/python2.7/site-packages/pyfits/hdu/image.py:515: VerifyWarning: Invalid 'BLANK' keyword in header.  The 'BLANK' keyword is only applicable to integer data, and will be ignored in this HDU.
  warnings.warn(msg, VerifyWarning)
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
No handlers could be found for logger "sewpy.sewpy"


40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLh010.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLh013.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLh014.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLs011.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLs012.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLs015.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLs016.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/z/IMG2016158BLs102.fits
Frame nr.  1 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/z/IMG2016158BLs103.fits
Frame nr.  2 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/z/IMG2016158BLs101.fits
10.456
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLh009.fits
40

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: Mean of empty slice


10.456
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLh009.fits
40.0
Frame nr.  1 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLh010.fits
40.0
Frame nr.  2 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLh013.fits
40.0
Frame nr.  3 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLh014.fits
40.0
Frame nr.  4 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLs011.fits
20.0
Frame nr.  5 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLs012.fits
20.0
Frame nr.  6 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLs015.fits
20.0
Frame nr.  7 /media/lynge/LACIE/Calibration/08-06-16-SA100-241/z/IMG2016158BLs016.fits
20.0
Frame nr.  0 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/z/IMG2016158BLs102.fits
Frame nr.  1 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/z/IMG2016158BLs103.fits
Frame nr.  2 /media/lynge/LACIE/Calibration/08-06-16-IC4329A/z/IMG2016158BLs101.fits


In [14]:
print 'z-filter'
print 'Star 1'
print com1_1
print np.nanmean((com1_1)), np.nanstd((com1_1))

print 'Star 2'
print com2_1
print np.nanmean((com2_1)), np.nanstd((com2_1))

print 'Star 3'
print com3_1
print np.nanmean((com3_1)), np.nanstd((com3_1))

z-filter
Star 1
[12.74873599 12.78243229 12.77173321]
12.767633830271597 0.014058537813173486
Star 2
[nan nan nan]
nan nan
Star 3
[14.57452703 14.57208598 14.60138604]
14.582666349803574 0.013274282488957501


/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  
/home/lynge/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [63]:
HD101274_1 = findmag(NGC3783_r,P_r,sta1_RA_WORLD,sta1_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)
Tyc_22_1 = findmag(NGC3783_r,P_r,sta2_RA_WORLD,sta2_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)
Tyc_529_1 = findmag(NGC3783_r,P_r,sta3_RA_WORLD,sta3_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)




NameError: name 'NGC3783_r' is not defined

In [ ]:
print 'r-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

In [ ]:
HD101274_1 = findmag(NGC3783_i,P_i,sta1_RA_WORLD,sta1_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)
Tyc_22_1 = findmag(NGC3783_i,P_i,sta2_RA_WORLD,sta2_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)
Tyc_529_1 = findmag(NGC3783_i,P_i,sta3_RA_WORLD,sta3_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)




In [ ]:
print 'i-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

In [ ]:
sta2_RA_WORLD = 174.82083333
sta2_DEC_WORLD = -37.76194444

HD101274_1 = findmag(NGC3783_z,P_z,sta1_RA_WORLD,sta1_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)
Tyc_22_1 = findmag(NGC3783_z,P_z,sta2_RA_WORLD,sta2_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)
Tyc_529_1 = findmag(NGC3783_z,P_z,sta3_RA_WORLD,sta3_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)




In [ ]:
print 'z-filter'
print HD101274_1
print np.nanmean(HD101274_1,), np.nanstd(HD101274_1)

print Tyc_22_1
print np.nanmean(Tyc_22_1), np.nanstd(Tyc_22_1)

print Tyc_529_1
print np.nanmean(Tyc_529_1), np.nanstd(Tyc_529_1)